## Descrição do Notebook

Este é um notebook somente para **EDA**. Os tratamentos feitos aqui foram feitos sobre os dados brutos iniciais e o *dataset* final gerado não foi utilizado como base na expansão dos dados.

In [127]:
import pandas as pd
import xlrd

from fuzzywuzzy import process
import numpy as np
import datetime

C:\Users\BlueShift\anaconda31\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


### Leitura  e observação do df principal

In [22]:
df_tratado = pd.read_csv('../datasets/df_rpo_v4_2.csv')
df_tratado.shape

(8107, 14)

### Checagem de duplicidade no df principal

In [23]:
df_tratado.duplicated(subset=['SAP+NOME', 'UF', 'Semana', 'Ano']).value_counts()

False    8107
dtype: int64

### Criação da coluna da variável dummy de preço psicologico

In [24]:
df_tratado['DPPsic'] = ''
df_tratado.loc[df_tratado['Preço'] != df_tratado['preço2'], 'DPPsic'] = 0
df_tratado.loc[df_tratado['Preço'] == df_tratado['preço2'], 'DPPsic'] = 1

### Filtro de ano da base

In [25]:
df_tratado2017 = df_tratado.loc[df_tratado['Ano'] == 2017]

### Calculo de correlação entre quantidade de vendas e preço

In [26]:
df_corr = df_tratado2017.groupby(['UF', 'SAP+NOME'])[['Preço', 'Vendas Reais Qtde']].corr().reset_index()

In [27]:
df_corr

,UF,SAP+NOME,level_2,Preço,Vendas Reais Qtde
0,AC,000000000002158151 - T VISITA P BORDA DESIRE A...,Preço,1.000000,-0.316228
1,AC,000000000002158151 - T VISITA P BORDA DESIRE A...,Vendas Reais Qtde,-0.316228,1.000000
2,AC,000000000002158154 - T VISITA P BORDA DESIRE A...,Preço,1.000000,-0.403187
3,AC,000000000002158154 - T VISITA P BORDA DESIRE A...,Vendas Reais Qtde,-0.403187,1.000000
4,AC,000000000002166665 - FRONHA AVULSA ESTAMPADA A...,Preço,1.000000,-0.330722
...,...,...,...,...,...
963,TO,000000000004269111 - DESINF LIQ PINHO SOL CITR...,Vendas Reais Qtde,NaN,NaN
964,TO,000000000004269808 - CIF ULTRA RAP BANH S/CLOR...,Preço,1.000000,-0.729667
965,TO,000000000004269808 - CIF ULTRA RAP BANH S/CLOR...,Vendas Reais Qtde,-0.729667,1.000000
966,TO,000000000004269883 - CIF ULT RAP TIRA LIMO C/C...,Preço,1.000000,-0.779193


In [28]:
df_corr.isnull().sum()

UF                    0
SAP+NOME              0
level_2               0
Preço                93
Vendas Reais Qtde    83
dtype: int64

In [29]:
df_corr = df_corr.loc[(df_corr['level_2'] == 'Vendas Reais Qtde') & (df_corr['Preço'].notnull()),\
                      ['UF', 'SAP+NOME','level_2','Preço']].sort_values(by='Preço', ascending=True).reset_index()

df_corr = df_corr[['UF', 'SAP+NOME', 'Preço']]

In [30]:
df_corr

,UF,SAP+NOME,Preço
0,PB,000000000004269111 - DESINF LIQ PINHO SOL CITR...,-1.000000
1,PA,000000000004269111 - DESINF LIQ PINHO SOL CITR...,-0.996779
2,BA,000000000004269883 - CIF ULT RAP TIRA LIMO C/C...,-0.983136
3,PA,000000000004269808 - CIF ULTRA RAP BANH S/CLOR...,-0.968319
4,SC,000000000004269883 - CIF ULT RAP TIRA LIMO C/C...,-0.962374
...,...,...,...
427,SC,000000000004270164 - DESINF LIQ PINHO SOL ORIG...,0.866025
428,RO,000000000004269111 - DESINF LIQ PINHO SOL CITR...,0.904534
429,SP,000000000004268705 - FRONHA AVULSA ESTAMPADA C...,0.923874
430,RS,000000000002168514 - FRONHA AVULSA LISA CAMESA,0.968562


### Cálculo do numero de semanas observadas por série

In [31]:
df_corr_obs = df_tratado2017.groupby(['UF', 'SAP+NOME']).Semana.nunique().reset_index()

In [32]:
df_corr_obs

,UF,SAP+NOME,Semana
0,AC,000000000002158151 - T VISITA P BORDA DESIRE A...,6
1,AC,000000000002158154 - T VISITA P BORDA DESIRE A...,17
2,AC,000000000002166665 - FRONHA AVULSA ESTAMPADA A...,11
3,AC,000000000002166719 - FRONHA AVULSA LISA ANDREZA,11
4,AC,000000000004243420 - AMACIANTE COMFORT CONCENT...,19
...,...,...,...
479,TO,000000000004267503 - PANO SCOTT DURAMAX LIMPEZ...,6
480,TO,000000000004268705 - FRONHA AVULSA ESTAMPADA C...,1
481,TO,000000000004269111 - DESINF LIQ PINHO SOL CITR...,2
482,TO,000000000004269808 - CIF ULTRA RAP BANH S/CLOR...,8


### DF com correlação entre preço e quantidade e número de semanas observadas por série

In [47]:
df_corr_final = pd.merge(df_corr_obs, df_corr, how='left', on=['UF', 'SAP+NOME'])
df_corr_final.rename(columns={"Preço":"Correlacao", "Semana":'NObs'}, inplace = True)
df_corr_final = df_corr_final.sort_values('NObs', ascending=False)
df_corr_final.reset_index(drop=True)

,UF,SAP+NOME,NObs,Correlacao
0,MS,000000000002158154 - T VISITA P BORDA DESIRE A...,44,-0.340838
1,DF,000000000002158154 - T VISITA P BORDA DESIRE A...,42,-0.421030
2,PE,000000000002158154 - T VISITA P BORDA DESIRE A...,42,-0.381781
3,PR,000000000002158151 - T VISITA P BORDA DESIRE A...,42,-0.480454
4,MA,000000000002158151 - T VISITA P BORDA DESIRE A...,41,-0.506632
...,...,...,...,...
479,PI,000000000004267503 - PANO SCOTT DURAMAX LIMPEZ...,1,NaN
480,MT,000000000004270164 - DESINF LIQ PINHO SOL ORIG...,1,NaN
481,MT,000000000004268705 - FRONHA AVULSA ESTAMPADA C...,1,NaN
482,AP,000000000004268705 - FRONHA AVULSA ESTAMPADA C...,1,NaN


In [56]:
### CSV para dashboard
#df_corr_final.to_csv('../datasets/db_corr_nobs.csv', decimal = ',', sep=';', index=False)

### Acrescenta o número de semanas observadas ao df principal, renomeia SAP+NOME

In [121]:
df_tratado_final = pd.merge(df_tratado2017, df_corr_final[['UF', 'SAP+NOME', 'NObs']], 
                            how='left', left_on=['UF', 'SAP+NOME'], right_on=['UF', 'SAP+NOME'])

df_tratado_final.rename(columns={'SAP+NOME':'Produto', 'Vendas Reais Qtde':'QTD'}, inplace = True)

sap_nome = df_tratado_final['Produto'].str.split(" - ", n=1, expand=True)

df_tratado_final['SAP'] = sap_nome[0].apply(lambda x: x.lstrip('0'))

df_tratado_final['Produto'] = sap_nome[1]

In [122]:
df_tratado_final

,Produto,UF,Semana,Ano,preço2,Mg %,Margem Líquida Real %,Preço,QTD,Vendas Reais $,Margem Líquida Real $,Venda Líquida Real $,CMV,CMV/QTD,DPPsic,NObs,SAP
0,T VISITA P BORDA DESIRE AZUL 6238,AL,1,2017.0,4.74,0.482330,48.233032,4.74000,7.0,33.93,12.29,24.73,12.44,1.777143,1,23,2158151
1,T VISITA P BORDA DESIRE AGUA 6243,AL,1,2017.0,4.99,0.535633,53.563281,4.99000,7.0,34.93,13.47,25.44,11.97,1.710000,1,27,2158154
2,AMACIANTE COMFORT CONCENTRADO 500ML,AL,1,2017.0,7.99,0.404144,40.414445,7.96619,99.0,790.01,289.63,716.93,427.30,4.316162,0,36,4243420
3,T VISITA P BORDA DESIRE AZUL 6238,AM,1,2017.0,4.74,0.587795,58.779525,4.74000,4.0,18.96,9.18,15.54,6.36,1.590000,1,36,2158151
4,T VISITA P BORDA DESIRE AGUA 6243,AM,1,2017.0,4.99,0.621958,62.195832,4.99000,15.0,74.85,38.05,61.35,23.30,1.553333,1,35,2158154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8102,T VISITA DISNEY CARS SANTISTA,TO,52,2017.0,4.99,0.457875,45.787546,4.99000,3.0,14.97,5.00,10.92,5.92,1.973333,1,15,4267279
8103,PANO SCOTT DURAMAX LIMPEZA DIARIA C/5,TO,52,2017.0,7.99,0.516757,51.675713,7.99000,4.0,31.96,12.02,23.27,11.25,2.812500,1,6,4267503
8104,FRONHA AVULSA ESTAMPADA CAMESA,TO,52,2017.0,4.99,0.516830,51.683022,4.99000,14.0,69.86,26.29,50.87,24.58,1.755714,1,1,4268705
8105,CIF ULTRA RAP BANH S/CLORO SQUEEZE 500ML,TO,52,2017.0,3.99,0.179620,17.962001,3.95000,14.0,55.06,7.07,40.11,33.04,2.360000,0,8,4269808


In [55]:
# df_tratado_final.to_csv('../datasets/df_tratado_final.csv', decimal = '.', sep=';', index=False)

### Criando df que será utilizado para ajustar o modelo - Produto individualizado

In [126]:
df_modelo_pi = df_tratado_final.loc[df_tratado_final['NObs'] >= 16, ['SAP', 'Produto', 'UF', 'Semana', 
                                                                   'Preço', 'QTD', 'CMV/QTD',
                                                                   'DPPsic']]

# SORTED()

In [159]:
aux = df_modelo_pi.copy()

lista_unicos = sorted(list(aux["Produto"].unique()))
aux["GrpProd"] = np.nan
grupo = 0
thresh = 52

# While

In [160]:
while len(lista_unicos) > 0:
    item = lista_unicos[0] 
    similaridades = process.extract(item, lista_unicos)
    for i in range(len(similaridades)):
        if (similaridades[i][1] >= thresh):
            aux.loc[aux["Produto"] == similaridades[i][0], "GrpProd"] = grupo
            lista_unicos.remove(similaridades[i][0])
    grupo += 1

## Validação

In [161]:
for i in range(len(aux["GrpProd"].unique())):
    print(f'grupo => {i}')
    print(aux.loc[aux["GrpProd"] == i]["Produto"].unique())

grupo => 0
['AMACIANTE COMFORT CONCENTRADO 500ML'
 'AMACIANTE DOWNY LIRIOS DO CAMPO 500ML'
 'AMACIANTE COMFORT PURO CUIDADO 500ML']
grupo => 1
['FRONHA AVULSA ESTAMPADA ANDREZA' 'FRONHA AVULSA LISA ANDREZA']
grupo => 2
['JG CAMA QUEEN PRATA 150FIOS MALIBU SANTI'
 'JG CAMA QUEEN MICROFIBRA ANDREZA']
grupo => 3
['T VISITA P BORDA DESIRE AGUA 6243' 'T VISITA DISNEY CARS SANTISTA'
 'T VISITA DISNEY BELA SANTISTA']
grupo => 4
['T VISITA P BORDA DESIRE AZUL 6238']


In [162]:
sorted(aux["GrpProd"].unique())[-1]

4.0

## Realizando os agrupamentos

In [163]:
def agrupamentos(dataframe, threshold:float):
    """
    Descreve uma função que classifica os itens em grupos de similaridade utilizando fuzzy matching.
    
    dataframe -> dataframe que contém os itens a serem classificados]
    threshold -> grau de similaridade entre os nomes dos itens (0 a  100)
    """
    
    # Cria um vetor de itens únicos ordenados alfabeticamente
    lista_unicos = sorted(list(dataframe["Produto"].unique()))
    
    # Cria uma feature de NaN's chamada Grupo
    dataframe["GrpProd"] = np.nan
    grupo = 0
    
    # Percorre o nosso dataframe e faz o agrupamento entre os itens
    while len(lista_unicos) > 0:
        item = lista_unicos[0] 
        similaridades = process.extract(item, lista_unicos)
        for i in range(len(similaridades)):
            if (similaridades[i][1] >= threshold):
                dataframe.loc[dataframe["Produto"] == similaridades[i][0], "GrpProd"] = grupo
                lista_unicos.remove(similaridades[i][0])
        grupo += 1
    
    return dataframe

### Criando df que será utilizado para ajustar o modelo - Produto agrupado

In [171]:
df_modelo_pa = agrupamentos(aux, 55)

In [173]:
aux['PrXQtd'] = aux['Preço'] * aux['QTD'] 

aux['CMVXQtd'] = aux['CMV/QTD'] * aux['QTD']

aux['PPsicXQtd'] = aux['DPPsic'] * aux['QTD']

aux['PPsicXQtd'] = aux['PPsicXQtd'].astype(int)

df_modelo_pa = aux[['GrpProd', 'UF', 'Semana', 'QTD', 
                   'PrXQtd', 'CMVXQtd', 'PPsicXQtd']].groupby(['GrpProd', 'UF', 'Semana']).sum().reset_index()
          
df_modelo_pa['Preço'] = df_modelo_pa['PrXQtd'] / df_modelo_pa['QTD']

df_modelo_pa['CMV/QTD'] = df_modelo_pa['CMVXQtd'] / df_modelo_pa['QTD']

df_modelo_pa['DPPsic'] = df_modelo_pa['PPsicXQtd'] / df_modelo_pa['QTD']

df_modelo_pa.drop(columns=['CMVXQtd', 'PrXQtd', 'PPsicXQtd'], inplace = True)

In [176]:
df_modelo_pa.head(60)

,GrpProd,UF,Semana,QTD,Preço,CMV/QTD,DPPsic
0,0.0,AC,3,10.0,7.990000,4.384000,1.000000
1,0.0,AC,13,7.0,4.990000,4.318571,1.000000
2,0.0,AC,14,17.0,4.990000,4.318235,1.000000
3,0.0,AC,25,4.0,7.990000,6.580000,1.000000
4,0.0,AC,27,5.0,8.990000,6.580000,1.000000
5,0.0,AC,31,12.0,7.990000,6.583333,1.000000
6,0.0,AC,32,10.0,7.990000,6.586000,1.000000
7,0.0,AC,33,24.0,8.634444,6.585833,0.000000
8,0.0,AC,35,12.0,7.990000,6.585000,1.000000
9,0.0,AC,37,4.0,8.990000,6.587500,1.000000


In [179]:
df_modelo_pa.to_csv('../datasets/df_modelo_pa.csv', decimal = '.', sep=';', index=False)

df_modelo_pi.to_csv('../datasets/df_modelo_pi.csv', decimal = '.', sep=';', index=False)